In [2]:
import csv
import numpy as np
from matplotlib import pyplot as plt 
import itertools

In [3]:
data_path = "Data/train.csv"

dtypes = "i8,S5,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,i8,f8,f8,f8,f8,f8,f8,f8"
with open(data_path, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader)
data = np.genfromtxt(data_path, delimiter=",",names=True, dtype=dtypes)

# REMOVING -999.0 values, turining to np.nan
data2 = data.copy()
nan_ind = []
for col in headers[2:]:
    for i in range(len(data2[col])):
        if data2[col][i] == -999.0:
            data2[col][i] = np.nan
            nan_ind.append(i)

nan_ind.sort()
nan_ind = list(set(nan_ind))

In [4]:
def split_by_thresholds(data2):
    #Identifiers
    split_d = {}

    split_d["PRI_jet_num"] = ["J0","J1","J2","J3"]
    split_d["DER_met_phi_centrality"] = ["p+","p-"]
    split_d["DER_lep_eta_centrality"] = ["e+","e-"] 

    # Expand jet_num to include combinations
    comb = list(itertools.combinations(split_d["PRI_jet_num"],1)) 
    comb = list(itertools.combinations(split_d["PRI_jet_num"],2)) + comb
    comb = list(itertools.combinations(split_d["PRI_jet_num"],3)) + comb
    split_d["PRI_jet_num"] = comb

    comb = split_d["PRI_jet_num"] + split_d["DER_met_phi_centrality"] + split_d["DER_lep_eta_centrality"]

    # Prepare combinations of 2 sets of features
    comb2 = list(itertools.product(*list(split_d.values())[0:2]))
    comb2 = list(itertools.product(*list(split_d.values())[1:3])) + comb2
    comb2 = list(itertools.product(*[list(split_d.values())[0],list(split_d.values())[-1]])) + comb2

    # Prepare combinations of 3 sets of features
    comb3 = list(itertools.product(*list(split_d.values())[0:3]))

    combination_names = comb+comb2+comb3
    
    #ACTUALLY GENERATING TRUTH TABLES
    # Select which header to split and how
    split_d = {}

    split_d["PRI_jet_num"] = [data2["PRI_jet_num"] == 0,
                              data2["PRI_jet_num"] == 1,
                              data2["PRI_jet_num"] == 2,
                              data2["PRI_jet_num"] == 3]

    split_d["DER_met_phi_centrality"] = [data2["DER_met_phi_centrality"] > 0,
                                         data2["DER_met_phi_centrality"] < 0]

    split_d["DER_lep_eta_centrality"] = [data2["DER_lep_eta_centrality"] > 0.5,
                                         data2["DER_lep_eta_centrality"] < 0.5]   


    # Expand jet_num to include combinations
    comb = list(itertools.combinations(split_d["PRI_jet_num"],1)) 
    comb = list(itertools.combinations(split_d["PRI_jet_num"],2)) + comb
    comb = list(itertools.combinations(split_d["PRI_jet_num"],3)) + comb
    comb = [np.logical_or.reduce(tup) for tup in comb]
    split_d["PRI_jet_num"] =  comb

    # Combinations of 1 set of features
    comb = split_d["PRI_jet_num"] + split_d["DER_met_phi_centrality"] + split_d["DER_lep_eta_centrality"]

    # Combinations of 2 sets of features
    comb2 = list(itertools.product(*list(split_d.values())[0:2]))
    comb2 = list(itertools.product(*list(split_d.values())[1:3])) + comb2
    comb2 = list(itertools.product(*[list(split_d.values())[0],list(split_d.values())[-1]])) + comb2
    comb2 = [np.logical_or.reduce(tup) for tup in comb2]

    # Combinations of 3 sets of features
    comb3 = list(itertools.product(*list(split_d.values())[0:3]))
    comb3 = [np.logical_or.reduce(tup) for tup in comb3]

    # All outputs
    combination = comb+comb2+comb3
    return combination, combination_names

In [5]:
def get_as_list_of_lists_training(xinput, extra_params=False):
    xlist=[]
    for i in xinput:
        sublist=[]
        for x in i:
            sublist.append(x)
        if extra_params:
            sublist=sublist[2:]       # Should be 1: if prediction is removed from the input data
        xlist.append(sublist)
        
    return xlist

In [6]:
def add_fake_feature(listx):
    for x in listx:
        x.insert(0,1)

In [7]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)

    # split the data based on the given ratio   
    p = np.random.permutation(len(y))
    y = y[p]
    x = x[p]
    
    limit = int(len(y)*ratio)
        
    return x[:limit],x[limit:],y[:limit],y[limit:]

In [8]:
def least_squares(y, tx):
    """calculate the least squares."""
    XX = np.dot(tx.transpose(),tx)
    B = np.dot(tx.transpose(),y)
    if len(XX.shape)==0:
        w_s = B/XX
    else:
        w_s = np.linalg.solve(XX,B)    
    
    loss = compute_loss(y, tx, w_s)
    
    return w_s, loss 

In [9]:
def compute_loss(y, tx, w):
    """Calculate the loss using MSE."""
    e = y - tx.dot(w)
    lv = 0.5*np.mean(np.square(e))
    return lv

In [12]:
# defining a function that executes the model

def execute_LPM(xtrainInput, ytrainInput, xtestInput, ytestInput):

    xlist = get_as_list_of_lists_training(xtestInput)
    add_fake_feature(xlist)

    trainxlist = get_as_list_of_lists_training(xtrainInput)
    add_fake_feature(trainxlist)

    # formatting y to array of 0,1
    #y=(ytrainInput==b's').astype(int)
    Ytrain=np.array(ytrainInput)
    Xtrain=np.array(trainxlist)
        
    weights,MSE=least_squares(Ytrain, Xtrain)
    
    #applying predictions
    xtest=np.array(xlist)

    if (len(weights.shape) == 0):
        return 0,0
    
    Ypred=np.dot(xtest,weights)
    Ypredclass=(Ypred>0.5).astype(int)
    
    for i in range(Ypredclass.size):
        if Ypredclass[i]==0:
            Ypredclass[i]=-1
            
    Ytest=np.array(ytestInput)
    n=len(Ytest)

    #test accuracy
    acctest=(Ypredclass==Ytest).sum()/n
    #train accuracy
    nt=Ytrain.size
    ypredtrain=np.dot(Xtrain,weights)
    ypredtrainclass=(ypredtrain>0.5).astype(int)
    acctrain=(Ytrain==ypredtrainclass).sum()/nt

    '''
    idcolumn=xtrainInput[['Id']]
    idlist=[i[0] for i in idcolumn]

    a=np.append(np.array(idlist).reshape(-1,1).astype(int),Ypredclass.reshape(-1,1).astype(int),axis=1)

    np.savetxt("submission.csv", a, delimiter=',', header="Id,Prediction", comments="", fmt='%d')
    '''
    
    return acctrain,acctest

In [13]:
comb, comb_n = split_by_thresholds(data2)

<ipython-input-4-1a3a3ecc6c31>:39: RuntimeWarning: invalid value encountered in greater
  split_d["DER_lep_eta_centrality"] = [data2["DER_lep_eta_centrality"] > 0.5,
<ipython-input-4-1a3a3ecc6c31>:40: RuntimeWarning: invalid value encountered in less
  data2["DER_lep_eta_centrality"] < 0.5]


In [14]:
ratio = 0.8 # Ratio of train data

for c_tr,c_n in zip(comb,comb_n):
    data_s = data2[c_tr]
    ylist = (data_s['Prediction']==b'b').astype(int)
    xlist = get_as_list_of_lists_training(data_s, True)
    
    Xtrain, Xtest, Ytrain, Ytest = split_data(np.array(xlist), ylist, ratio, seed=1)
    
    nans = {}
    
    for point in Xtrain:
        for i,feat in enumerate(point):
            if np.isnan(feat):
                if i in nans:
                    nans[i] += 1
                else:
                    nans[i] = 1
                
    print(c_n, nans)

KeyboardInterrupt: 

In [17]:
# Deletes all points that contain NaNs
# Issue: causes matrix to shrink considerably, making it singular

def delete_nan_points(x,y):
    ret_x, ret_y = zip(*[(x_point,y_point) for x_point,y_point in zip(x,y) if not np.isnan(x_point).any()])
    return ret_x, ret_y

In [113]:
#b = np.concatenate(data2).reshape(len(data2),*np.shape(a[data2]))
#b = np.concatenate(data2).reshape(len(data2),*np.shape(np.array(list(data2[0]))))

#np.apply_along_axis(lambda p: np.array(list(p)), 0, data2)[0]

#z = np.zeros(shape=(data2.shape[0], len(data2[0])))
#z = np.apply_along_axis(lambda p: np.array(list(p)), 0, z)

d = np.array(get_as_list_of_lists_training(data2, True))

In [154]:
#np.apply_along_axis(lambda f: np.apply_along_axis(lambda p: np.median(f), 0, f), 1, d)

for col in range(d.shape[1]):
    m = np.nanmedian(d[:,col])
    thingy = lambda p: p if not np.isnan(p) else m
    vfunc = np.vectorize(thingy)
    d[:,col] = vfunc(d[:,col])

In [156]:
def replace_nans_with_median(x):
    for col in range(x.shape[1]):
        m = np.nanmedian(x[:,col])
        nan_to_median = lambda p: p if not np.isnan(p) else m
        vfunc = np.vectorize(nan_to_median)
        x[:,col] = vfunc(x[:,col])

In [25]:
ratio = 0.8 # Ratio of train data

for c_tr,c_n in zip(comb,comb_n):
    data_s = data2[c_tr]
    ylist = (data_s['Prediction']==b'b').astype(int)
    xlist = get_as_list_of_lists_training(data_s, True)
    
    Xtrain, Xtest, Ytrain, Ytest = split_data(np.array(xlist), ylist, ratio, seed=1)
    
    Xtest, Ytest = delete_nan_points(Xtest,Ytest)
    Xtrain, Ytrain = delete_nan_points(Xtrain,Ytrain)
                
    if len(Xtrain) == 0 or len(Xtest) == 0:
        print(c_n, "No operation can be performed, as all points have NaNs")
        continue
    
    acctrain,acctest = execute_LPM(Xtrain, Ytrain, Xtest, Ytest)
    
    print(c_n, acctest, acctrain)

182268 182268
45568 45568
30


LinAlgError: Singular matrix

In [ ]:
data2[comb[0]]

In [ ]:
data2[comb[0]][2000]

In [ ]:
Xtrain.shape

In [ ]:
Ytrain.shape

In [ ]:
Ytest.shape